In [37]:
import cv2
import numpy as np
from smoothing import gaussian_smoothing_kernel
from sharpening import sharpening_kernel
from convolution import apply_convolution
import matplotlib.pyplot as plt

In [38]:
lena = cv2.imread('./Lena.jpg', 1)

cv2.imshow('Original Image', lena)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.a: Take an RGB image and apply convolution with each kernel, separately on each channel.


In [39]:
lena_b = lena.copy()
lena_b[:, :, 1] = 0
lena_b[:, :, 2] = 0
print(lena_b.shape)

lena_g = lena.copy()
lena_g[:, :, 0] = 0
lena_g[:, :, 2] = 0
print(lena_g.shape)

lena_r = lena.copy()
lena_r[:, :, 0] = 0
lena_r[:, :, 1] = 0
print(lena_r.shape)


(512, 512, 3)
(512, 512, 3)
(512, 512, 3)


In [40]:
cv2.imshow("Lena Blue Channel", lena_b)
cv2.imshow("Lena Green Channel", lena_g)
cv2.imshow("Lena Red Channel", lena_r)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
bc, gc, rc = cv2.split(lena)

print(bc.shape, gc.shape, rc.shape)

(512, 512) (512, 512) (512, 512)


- Gaussian Smoothing Kernel

In [42]:
smk = gaussian_smoothing_kernel(5, 1.67)

print(smk)

smooth_bc = apply_convolution(bc, smk)
smooth_gc = apply_convolution(gc, smk)
smooth_rc = apply_convolution(rc, smk)

[[0.00687486 0.02015726 0.02885054 0.02015726 0.00687486]
 [0.02015726 0.05910157 0.08459048 0.05910157 0.02015726]
 [0.02885054 0.08459048 0.12107208 0.08459048 0.02885054]
 [0.02015726 0.05910157 0.08459048 0.05910157 0.02015726]
 [0.00687486 0.02015726 0.02885054 0.02015726 0.00687486]]


In [43]:
# sobel_1 = np.array(
#     [
#         [-1, 0, 1],
#         [-1, 0, 1],
#         [-1, 0, 1]
#     ]
# )

# sobel_2 = np.array(
#     [
#         [-1, -1, -1],
#         [0, 0, 0],
#         [1, 1, 1]
#     ]
# )

# apply_sobel_1 = apply_convolution(bc, sobel_1)
# apply_sobel_2 = apply_convolution(bc, sobel_2)

# sb1_cropped = apply_sobel_1["final"]
# sb2_cropped = apply_sobel_2["final"]


# gradient_magnitude = np.sqrt(sb1_cropped.astype(np.float64)**2 + sb2_cropped.astype(np.float64)**2)

# # Normalize to 0-255 range for display
# gradient_magnitude_norm = np.clip(gradient_magnitude, 0, 255).astype(np.uint8)
# test = 255-gradient_magnitude_norm

# cv2.imshow("Sobel 1", sb1_cropped)
# cv2.imshow("Sobel 2", sb2_cropped)
# cv2.imshow("Gradient Magnitude", test)



# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [44]:
border_img_bc = smooth_bc["border_img"]
convo_bc = smooth_bc["raw_result"]
norm_bc = smooth_bc["normalized"]
norm_cropped_bc = smooth_bc["final"]

border_img_gc = smooth_gc["border_img"]
convo_gc = smooth_gc["raw_result"]
norm_gc = smooth_gc["normalized"]
norm_cropped_gc = smooth_gc["final"]

border_img_rc = smooth_rc["border_img"]
convo_rc = smooth_rc["raw_result"]
norm_rc = smooth_rc["normalized"]
norm_cropped_rc = smooth_rc["final"]

intermediate = cv2.merge((norm_cropped_bc, norm_cropped_gc, norm_cropped_rc))
final = np.round(cv2.normalize(intermediate, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [45]:
blue_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
blue_only[:, :, 0] = 255 
blue_only[:, :, 1] = 255-norm_cropped_bc
blue_only[:, :, 2] = 255-norm_cropped_bc

green_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
green_only[:, :, 0] = 255-norm_cropped_gc
green_only[:, :, 1] = 255
green_only[:, :, 2] = 255-norm_cropped_gc

red_only = np.zeros((norm_cropped_bc.shape[0], norm_cropped_bc.shape[1], 3), dtype=np.uint8)
red_only[:, :, 0] = 255-norm_cropped_rc
red_only[:, :, 1] = 255-norm_cropped_rc
red_only[:, :, 2] = 255

test = cv2.merge((norm_cropped_bc, norm_cropped_gc, norm_cropped_rc))

cv2.imshow("Blue Channel Convolution Only", blue_only)
cv2.imshow("Green Channel Convolution Only", green_only)
cv2.imshow("Red Channel Convolution Only", red_only)
cv2.imshow("Final Merged Image", test)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [46]:
cv2.imshow("Blue Channel Border", border_img_bc)
cv2.imshow("Blue Channel Convo", convo_bc)
cv2.imshow("Blue Channel Norm", norm_bc)
cv2.imshow("Blue Channel Norm Cropped", norm_cropped_bc)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
cv2.imshow("Green Channel Border", border_img_gc)
cv2.imshow("Green Channel Convo", convo_gc)
cv2.imshow("Green Channel Norm", norm_gc)
cv2.imshow("Green Channel Norm Cropped", norm_cropped_gc)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
cv2.imshow("Red Channel Border", border_img_rc)
cv2.imshow("Red Channel Convo", convo_rc)
cv2.imshow("Red Channel Norm", norm_rc)
cv2.imshow("Red Channel Norm Cropped", norm_cropped_rc)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
cv2.imshow("Real", lena)
cv2.imshow("Final Merged Image", final)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Gaussian Sharpening (LoG) Kernel

In [50]:
bc2, gc2, rc2 = cv2.split(lena)

In [51]:
shk = sharpening_kernel(7, 1.67)
sh_bc = apply_convolution(bc2, shk)
sh_gc = apply_convolution(gc2, shk)
sh_rc = apply_convolution(rc2, shk)

In [52]:
border_img_bc_2 = sh_bc["border_img"]
convo_bc_2 = sh_bc["raw_result"]
norm_bc_2 = sh_bc["normalized"]
norm_cropped_bc_2 = sh_bc["final"]

border_img_gc_2 = sh_gc["border_img"]
convo_gc_2 = sh_gc["raw_result"]
norm_gc_2 = sh_gc["normalized"]
norm_cropped_gc_2 = sh_gc["final"]

border_img_rc_2 = sh_rc["border_img"]
convo_rc_2 = sh_rc["raw_result"]
norm_rc_2 = sh_rc["normalized"]
norm_cropped_rc_2 = sh_rc["final"]

int2 = cv2.merge((norm_cropped_bc_2, norm_cropped_gc_2, norm_cropped_rc_2))
final2 = np.round(cv2.normalize(int2, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [83]:
blue_only_2 = np.zeros((norm_cropped_bc_2.shape[0], norm_cropped_bc_2.shape[1], 3), dtype=np.uint8)
blue_only_2[:, :, 0] = norm_cropped_bc_2-255      
blue_only_2[:, :, 1] =  0-norm_cropped_gc_2-255 
blue_only_2[:, :, 2] = 0-norm_cropped_rc_2-255  

green_only_2 = np.zeros((norm_cropped_gc_2.shape[0], norm_cropped_gc_2.shape[1], 3), dtype=np.uint8)
green_only_2[:, :, 0] = 255-norm_cropped_gc_2 
green_only_2[:, :, 1] = norm_cropped_gc_2        
green_only_2[:, :, 2] = 255 -norm_cropped_gc_2  

red_only_2 = np.zeros((norm_cropped_rc_2.shape[0], norm_cropped_rc_2.shape[1], 3), dtype=np.uint8)
red_only_2[:, :, 0] = 255- norm_cropped_rc_2    
red_only_2[:, :, 1] = 255- norm_cropped_rc_2   
red_only_2[:, :, 2] = norm_cropped_rc_2         

final_2 = cv2.merge((norm_cropped_bc_2, norm_cropped_gc_2, norm_cropped_rc_2))

cv2.imshow("Blue Channel Convolution Only", blue_only_2)
cv2.imshow("Green Channel Convolution Only", green_only_2)
cv2.imshow("Red Channel Convolution Only", red_only_2)
cv2.imshow("Final Merged Image", final_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [54]:
cv2.imshow("Blue Channel Border", border_img_bc_2)
cv2.imshow("Blue Channel Convo", convo_bc_2)
cv2.imshow("Blue Channel Norm", norm_bc_2)
cv2.imshow("Blue Channel Norm Cropped", norm_cropped_bc_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [55]:
cv2.imshow("Green Channel Border", border_img_gc_2)
cv2.imshow("Green Channel Convo", convo_gc_2)
cv2.imshow("Green Channel Norm", norm_gc_2)
cv2.imshow("Green Channel Norm Cropped", norm_cropped_gc_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [56]:
cv2.imshow("Red Channel Border", border_img_rc_2)
cv2.imshow("Red Channel Convo", convo_rc_2)
cv2.imshow("Red Channel Norm", norm_rc_2)
cv2.imshow("Red Channel Norm Cropped", norm_cropped_rc_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
cv2.imshow("Real", lena)
cv2.imshow("Final Merged Image", final2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.b : Convert the RGB image to HSV mode and apply convolution with each kernel, separately on each channel of HSV space.

In [58]:
lena_hsv = cv2.cvtColor(lena, cv2.COLOR_BGR2HSV)

cv2.imshow("Lena HSV", lena_hsv)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
h, s, v = cv2.split(lena_hsv)

print(h.shape)
print(s.shape)
print(v.shape)

lena_h = lena_hsv.copy()
lena_h[:, :, 1] = 0
lena_h[:, :, 2] = 0

lena_s = lena_hsv.copy()
lena_s[:, :, 0] = 0
lena_s[:, :, 2] = 0

lena_v = lena_hsv.copy()
lena_v[:, :, 0] = 0
lena_v[:, :, 1] = 0

cv2.imshow("Lena H Channel", lena_h)
cv2.imshow("Lena S Channel", lena_s)
cv2.imshow("Lena V Channel", lena_v)

cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512)
(512, 512)
(512, 512)


- Gaussian Smoothing Kernel

In [60]:
sm_kernel = gaussian_smoothing_kernel(5, 1.67)

h_sm_convo = apply_convolution(h, sm_kernel)
s_sm_convo = apply_convolution(s, sm_kernel)
v_sm_convo = apply_convolution(v, sm_kernel)

In [61]:
h_border = h_sm_convo["border_img"]
h_raw = h_sm_convo["raw_result"]
h_norm = h_sm_convo["normalized"]
h_final = h_sm_convo["final"]

s_border = s_sm_convo["border_img"]
s_raw = s_sm_convo["raw_result"]
s_norm = s_sm_convo["normalized"]
s_final = s_sm_convo["final"]

v_border = v_sm_convo["border_img"]
v_raw = v_sm_convo["raw_result"]
v_norm = v_sm_convo["normalized"]
v_final = v_sm_convo["final"]


int3 = cv2.merge((h_final, s_final, v_final))
final_sm = np.round(cv2.normalize(int3, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [62]:
cv2.imshow("H Border", h_border)
cv2.imshow("H Raw", h_raw)
cv2.imshow("H Norm", h_norm)
cv2.imshow("H Final", h_final)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
cv2.imshow("S Border", s_border)
cv2.imshow("S Raw", s_raw)
cv2.imshow("S Norm", s_norm)
cv2.imshow("S Final", s_final)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [64]:
cv2.imshow("V Border", v_border)
cv2.imshow("V Raw", v_raw)
cv2.imshow("V Norm", v_norm)
cv2.imshow("V Final", v_final)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
cv2.imshow("Original HSV", lena_hsv)
cv2.imshow("Final SM", final_sm)

cv2.waitKey(0)
cv2.destroyAllWindows()

- Sharpening Kernel

In [66]:
sh_kernel = sharpening_kernel(7, 1.67)

sh_h_convo = apply_convolution(h, sh_kernel)
sh_s_convo = apply_convolution(s, sh_kernel)
sh_v_convo = apply_convolution(v, sh_kernel)

In [67]:
h_border_sh = sh_h_convo["border_img"]
h_raw_sh = sh_h_convo["raw_result"]
h_norm_sh = sh_h_convo["normalized"]
h_final_sh = sh_h_convo["final"]

s_border_sh = sh_s_convo["border_img"]
s_raw_sh = sh_s_convo["raw_result"]
s_norm_sh = sh_s_convo["normalized"]
s_final_sh = sh_s_convo["final"]

v_border_sh = sh_v_convo["border_img"]
v_raw_sh = sh_v_convo["raw_result"]
v_norm_sh = sh_v_convo["normalized"]
v_final_sh = sh_v_convo["final"]

int4 = cv2.merge((h_final_sh, s_final_sh, v_final_sh))
final_sh = np.round(cv2.normalize(int4, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [68]:
cv2.imshow("H Border", h_border_sh)
cv2.imshow("H Raw", h_raw_sh)
cv2.imshow("H Norm", h_norm_sh)
cv2.imshow("H Final", h_final_sh)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [69]:
cv2.imshow("S Border", s_border_sh)
cv2.imshow("S Raw", s_raw_sh)
cv2.imshow("S Norm", s_norm_sh)
cv2.imshow("S Final", s_final_sh)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [70]:
cv2.imshow("V Border", v_border_sh)
cv2.imshow("V Raw", v_raw_sh)
cv2.imshow("V Norm", v_norm_sh)
cv2.imshow("V Final", v_final_sh)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
cv2.imshow("Original HSV", lena_hsv)
cv2.imshow("Final SH", final_sh)

cv2.waitKey(0)
cv2.destroyAllWindows()